Dar uma olhada nesses links:

(grafico de cauda pintado em certas áreas)
https://www.geo.fu-berlin.de/en/v/soga-py/Basics-of-statistics/Continous-Random-Variables/Students-t-Distribution/Students-t-Distribution-in-Python/index.html


# T-Student

### Comparação de duas médias populacionais a partir de duas amostras aleatórias independentes

O teste t para duas amostras independentes é aplicado para comparar as médias de duas amostras aleatórias extraídas da mesma população.



Para um teste bilateral,

$$H_0 : \mu _1 = \mu _2 \\
H_1 : \mu _1 \neq \mu _2$$




Caso 1:

$\sigma^2 _1 \neq \sigma^2 _2$

Variâncias populacionais homogêneas.

Considerando que as variâncias populacionais são diferentes, a estatística T é dada por:

$T_{calc}$ = $\frac{ \bar{X_1}-\bar{X_2} }{\sqrt{}}$


Onde $S^2 _1\: e\: S^2 _2$ são as variâncias amostrais



In [105]:
import numpy as np
import random
import pandas as pd
from scipy.stats import norm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sympy import latex

In [55]:
try:
    t_student_table = pd.read_excel(open('Tabelas.xlsx','rb'),sheet_name='t-Student',header=6)
except Exception as e:
    print("Ocorreu um erro ao ler o arquivo XLSX",e)

In [56]:
t_student_table = t_student_table.dropna(axis=1) #dropa colunas NaN

t_student_table = t_student_table.rename(columns={'Liberdade':'graus_liberdade'})


for c in (t_student_table.columns[1:]):
  t_student_table = t_student_table.rename(columns={c:'alpha = ' + str(c)})

t_student_table

,graus_liberdade,alpha = 0.01,alpha = 0.02,alpha = 0.05,alpha = 0.1,alpha = 0.2
0,1,63.656741,31.820516,12.706205,6.313752,3.077684
1,2,9.924843,6.964557,4.302653,2.919986,1.885618
2,3,5.840909,4.540703,3.182446,2.353363,1.637744
3,4,4.604095,3.746947,2.776445,2.131847,1.533206
4,5,4.032143,3.364930,2.570582,2.015048,1.475884
...,...,...,...,...,...,...
69,400,2.588176,2.335706,1.965912,1.648672,1.283672
70,450,2.586799,2.334663,1.965250,1.648247,1.283436
71,500,2.585698,2.333829,1.964720,1.647907,1.283247
72,1000,2.580755,2.330083,1.962339,1.646379,1.282399


Criação das amostras independentes

In [65]:
# Coletar dados sobre o tempo de fabricação de peças de carro em 2 linhas de produção diferentes
random.seed(0)
tempo_linha1  = np.random.normal(20,3,20)
tempo_linha2  = np.random.normal(19,4,20)

In [66]:
tempo_linha1.mean()

18.442030498483103

Estatísticas das amostras para serem utilizadas nas fórmulas de Tcalc e do número de graus de liberdade

In [67]:
def statistics_amostra(amostra):
  return amostra.mean(), amostra.var(), len(amostra)

### Tcalc

In [68]:
def tcalc(amostra1, amostra2):
  X_bar_1, var_amostral_1, n1 = statistics_amostra(amostra1)
  X_bar_2, var_amostral_2, n2 = statistics_amostra(amostra2)


  Tcalc = np.absolute(X_bar_1-X_bar_2)/np.sqrt((var_amostral_1/n1)+(var_amostral_2/n2))

  return Tcalc

In [69]:
tcalc(tempo_linha1,tempo_linha2)

0.08318669666200311

Número de graus de liberdade

In [70]:
def graus_liberdade(amostra1,amostra2):
  var_amostral_1, n1 = statistics_amostra(amostra1)[1:]
  var_amostral_2, n2 = statistics_amostra(amostra2)[1:]

  var_1_sobre_n1, var_2_sobre_n2 =  var_amostral_1/n1, var_amostral_2/n2

  v = np.power(np.sum((var_1_sobre_n1,var_2_sobre_n2)),2)/ ((var_1_sobre_n1)**2/(n1-1) + (var_2_sobre_n2)**2/(n2-1))

  return v

In [76]:
graus_liberdade(tempo_linha1,tempo_linha2)

34.0

T_crit

In [96]:
def t_critico(amostra1,amostra2):
  v = graus_liberdade(amostra1,amostra2).round()

  t_critico = t_student_table[t_student_table['graus_liberdade'] == int(v)]['alpha = 0.05']
  
  return t_critico.values[0]


In [82]:
i = t_student_table[t_student_table['graus_liberdade'] == 31]['alpha = 0.05']
i.values[0]

2.039513446396408

In [97]:
t_critico(tempo_linha1,tempo_linha2)

2.032244509317719

Verificação da hipótese

In [98]:
def verify_hipotese(amostra1,amostra2):
    T_crit = t_critico(amostra1, amostra2)
    T_calc = tcalc(amostra1, amostra2)

    if T_calc < T_crit:
        return True #Hipótese nula (H_0) é aceita
    else:
        return False


In [99]:
verify_hipotese(tempo_linha1,tempo_linha2)

True

In [110]:
if verify_hipotese(tempo_linha1,tempo_linha2):
    print('Hipótese nula aceita.')
    print('As médias populacionais são iguais')

else:
    print('Hipótese nula rejeitada')

Hipótese nula aceita.
As médias populacionais são iguais


### Comparação de duas médias populacionais a partir de duas amostras aleatórias emparelhadas

$$H_0:\mu _d =0, \\
H_1:\mu _d \neq 0.\\
\mu _d = \mu _{antes} - \mu _{depois}$$

Cenário:

Notas de redação antes e depois da preparação de um curso de redação

In [186]:
np.random.seed(1)

notas_antes  = np.random.normal(loc=600, scale=100, size=40)
notas_antes  = notas_antes.round(-1)

notas_depois = np.random.normal(loc=800, scale=120, size=40)
notas_depois = notas_depois.round(-1) 

In [187]:

notas_antes

array([760., 540., 550., 490., 690., 370., 770., 520., 630., 580., 750.,
       390., 570., 560., 710., 490., 580., 510., 600., 660., 490., 710.,
       690., 650., 690., 530., 590., 510., 570., 650., 530., 560., 530.,
       520., 530., 600., 490., 620., 770., 670.])

In [193]:
data = {'Estudante': [f'Estudante{i+1}' for i in range(len(notas_antes))],
        'X_antes_i': notas_antes.astype(int),  
        'X_depois_i': notas_depois.astype(int)} 

df_emparelhado = pd.DataFrame(data)
df_emparelhado.head()

,Estudante,X_antes_i,X_depois_i
0,Estudante1,760,780
1,Estudante2,540,690
2,Estudante3,550,710
3,Estudante4,490,1000
4,Estudante5,690,810


In [194]:
df_emparelhado['d_i'] = df_emparelhado['X_antes_i'] - df_emparelhado['X_depois_i']

In [196]:
df_emparelhado.head()

,Estudante,X_antes_i,X_depois_i,d_i
0,Estudante1,760,780,-20
1,Estudante2,540,690,-150
2,Estudante3,550,710,-160
3,Estudante4,490,1000,-510
4,Estudante5,690,810,-120


In [197]:
def calculate_d_bar(d_i,n):
    d_bar = d_i.sum()/n
    return d_bar

In [201]:
d_bar = calculate_d_bar(df_emparelhado['d_i'],40)

In [202]:
df_emparelhado['(d_i - d_bar)^2'] = (df_emparelhado['d_i'] - d_bar)**2

In [204]:
df_emparelhado.head()

,Estudante,X_antes_i,X_depois_i,d_i,(d_i - d_bar)^2
0,Estudante1,760,780,-20,44838.0625
1,Estudante2,540,690,-150,6683.0625
2,Estudante3,550,710,-160,5148.0625
3,Estudante4,490,1000,-510,77423.0625
4,Estudante5,690,810,-120,12488.0625


In [206]:
Sd = np.sqrt((df_emparelhado['(d_i - d_bar)^2'].sum())/(40-1))

In [209]:
Tcalc = (d_bar -( df_emparelhado['X_antes_i'].mean() - df_emparelhado['X_depois_i'].mean())) / (Sd/np.sqrt(40))

In [211]:
(d_bar -( df_emparelhado['X_antes_i'].mean() - df_emparelhado['X_depois_i'].mean()))

0.0

In [210]:
Tcalc

0.0

In [223]:
df_emparelhado['X_antes_i'].var()
#df_emparelhado['X_antes_i'].max()

9527.948717948719

0    3.583333
dtype: float64

grau de liberdade
df = n - 1

### Scipy